# **Import necesarry modules**

In [67]:
import numpy as np
import copy as cp
import sympy as sp

# **Functions**
## **List**
Number theoretic transform(NTT) <br>
Inverse number theoretic transform(INTT) <br>
# **Comments**
## **Number theoretic transform(NTT)**
This function is implemented by an algorithm, "Function NTT based on the Cooley-Tukey (CT) butterfly" <br>
## **Inverse number theoretic transform(INTT)**
This function is implemented by an, algorithm, "Function INTT based on the Gentleman-Sande (GS) butterfly" <br>



In [79]:
class NTT:     
    def __init__(self, polynomial, q):
        #Input data: polynomial(list), q(integer)
        #Although the input data is list, we treat it as a polynomial.
        #q is modulo
        self.polynomial = polynomial
        self.degree = len(polynomial)
        self.modulo = q
        
        #Generate twiddle factors
        #Step 1) Find generator
        n = cp.copy(self.degree)
        generator = 0 #Initial value
        test_value = 2
        while generator == 0:
            if (test_value**(2*n))%q != 1: #Does test_value is a 2n root of unity?
                test_value += 1
                continue
            else:
                pass #Yes, test_value is a 2n root of unity.
            
            test = 0 #Test for 2, 3, ..., 2n-1
            for i in range(2, 2*n):
                if (test_value**i)%q == 1:
                    test = 1 #test_value is not the 2n-primitive root of unity
                    continue
                else:
                    pass
            
            if test == 1: #For doing next test
                test_value += 1
                continue
            else:
                pass
            
            generator = test_value  
        #Step 2) Gnerate psi and psi_inverse vector
        psi = []
        psi_inverse = []
        for i in range(0, n):
            psi.append((generator**i)%q)
            psi_inverse.append((generator**(2*n-i))%q)
        self.psi = psi
        self.psi_inverse = psi_inverse
        
    def introduce(self):
        #Just for testing
        print("Polynomial: ", self.polynomial)
        print("Degree: ", self.degree)
        print("Modulo: ", self.modulo)
        print("Psi: ", self.psi)
        print("Psi_inverse: ", self.psi_inverse)
        print("Polynomial after NTT: ", self.ntt)
        print("Polynomial after INTT: ", self.intt)
        
    def CP(self):
        #Check parameters
        
        #Does degree is a power of 2?
        N = cp.copy(self.degree) #degree
        while N > 1: 
            if N%2 == 1:
                break
                print("The degree of the given polynomial is not a power of 2.")
            else:
                N = N/2
        
        #Does modulo is suitable?
        q = cp.copy(self.modulo)
        
        if q%int(q) != 0 or q <= 0: #Check whether q is a positive integer or not
            print("q is not a positive integer")
        else:
            pass
        
        if q%(2*self.degree) == 1: #Check q (mod 2n) = 1
            pass
        else:
            print("q mod 2n != 1")
        
        if q == 1: #Check whether q is a prime number or not
            print("modulo is not a prime number")
        elif q > 1:
            for i in range(2, q):
                if q%i == 0:
                    break
                    print("q is not a prime number")
                else:
                    pass   
    
    def ntt(self):
        a = cp.copy(self.polynomial)
        q = cp.copy(self.modulo)
        n = cp.copy(self.degree)
        t = cp.copy(self.degree)
        m = 1
        while m < n:
            t = t//2
            for i in range(0, m):
                j_1 = 2*i*t
                j_2 = j_1 + t - 1
                s = cp.copy(self.psi[m+i])
                for j in range(j_1, j_2 + 1):
                    u = a[j]
                    v = a[j+t]*s
                    a[j] = (u+v)%q
                    a[j+t] = (u-v)%q
            m = 2*m
        self.ntt = a
    
    def intt(self):
        a = cp.copy(self.ntt)
        n = cp.copy(self.degree)
        q = cp.copy(self.modulo)
        t = 1
        m = n
        while m > 1:
            j_1 = 0
            h = m//2
            for i in range(0, h):
                j_2 = j_1 + t - 1
                s = cp.copy(self.psi_inverse[h+i])
                for j in range(j_1, j_2 + 1):
                    u = a[j]
                    v = a[j+t]
                    a[j] = (u+v)%q
                    a[j+t] = ((u-v)*s)%q
                j_1 += 2*t
            t = 2*t
            m = m//2
        for j in range(0, n):
            a[j] = (a[j]/n)%q
        self.intt = a
    
        
    def encode(self):
        #Pointwise multiplication between polynomial and psi
        encoded = []
        polynomial = cp.copy(self.polynomial)
        psi = cp.copy(self.psi)
        for i in range(0, n):
            encoded.append(polynomial[i]*psi[i])
            
        return encoded
    
    def naive_ntt(self):
        a = cp.copy(self.polynomial)
        n = cp.copy(self.degree)
        q = cp.copy(self.modulo)
        psi = cp.copy(self.psi)
        omega = cp.copy(self.psi[2])
        #Encode part
        encoded = []
        for i in range(0, n):
            encoded.append(a[i]*psi[i])
        
        #NTT part
        result = [0]*n
        for i in range(0, n):
            for j in range(0, n):
                result[i] = result[i] + encoded[j]*(omega**(i*j))
            result[i] = result[i]%q
        return result
        
    
        

In [78]:
a = NTT([0,1,2,3,4,5,6,7], 17)
a.ntt()
a.intt()
a.introduce()
a.naive_ntt()

Polynomial:  [0, 1, 2, 3, 4, 5, 6, 7]
Degree:  8
Modulo:  17
Psi:  [1, 3, 9, 10, 13, 5, 15, 11]
Psi_inverse:  [1, 6, 2, 12, 4, 7, 8, 14]
Polynomial after NTT:  [12, 15, 5, 16, 12, 1, 2, 5]
Polynomial after INTT:  [0.0, 1.0, 2.0, 0.875, 1.875, 0.75, 1.75, 0.625]


[6, 13, 5, 12, 8, 0, 2, 5]

In [68]:
sp.discrete.transforms.ntt([0,1,2,3,4,5,6,7], 17)

AttributeError: module 'sympy' has no attribute 'discrete'